In [1]:
!pip install -U "pandas" "indic-nlp-library" "transformers[torch]" "datasets" "httpx==0.24.0" "accelerate>=0.26.0" "scikit-learn"
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git indic_nlp_resources

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 103.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 140.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully 

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126 (from 1)
Receiving objects: 100% (139/139), 149.77 MiB | 35.69 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [1]:
!pip install indic-nlp-library -q
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

from indicnlp import common
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
import pandas as pd
from sklearn.model_selection import train_test_split
import re
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer
from datasets import Dataset
import torch
import os

# Set up IndicNLP resources
common.set_resources_path("./indic_nlp_resources")
factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("ta")

print("Dependencies and resources set up successfully!")

fatal: destination path 'indic_nlp_resources' already exists and is not an empty directory.
Dependencies and resources set up successfully!


In [2]:
import pandas as pd

# Load with UTF-8 (try this first)
df = pd.read_csv('/content/Tamil-News-Headlines.csv', encoding='utf-8')

# Check if Tamil text is correctA
print("First Tamil news text:")
print(df['News'].iloc[0])

# If garbled, try latin-1 and save fixed version
if 'à' in df['News'].iloc[0]:  # Check for garbled chars
    df_latin1 = pd.read_csv('/content/Tamil-News-Headlines.csv', encoding='latin-1')
    df_latin1.to_csv('fixed_encoding.csv', encoding='utf-8', index=False)
    print("Saved fixed file as 'fixed_encoding.csv'")
    df = df_latin1  # Use fixed data

# Save for next steps
df.to_csv('dataset_ready.csv', index=False, encoding='utf-8')
print("Dataset ready!")

First Tamil news text:
 பாஸ்வேர்டை பகிரும் பயனர்களிடம் கூடுதல் கட்டணம்: நெட்ஃப்ளிக்ஸ் பலே திட்டம்  
Dataset ready!


In [3]:

# Set up IndicNLP for Tamil normalization
resources_path = "./indic_nlp_resources"
# The resource cloning is already done in cell 07376836, no need to check or clone again
common.set_resources_path(resources_path)
factory = IndicNormalizerFactory()
normalizer = factory.get_normalizer("ta")

# Load data with error handling for encoding
try:
    df = pd.read_csv('dataset_ready.csv', encoding='utf-8')
except UnicodeDecodeError:
    df = pd.read_csv('dataset_ready.csv', encoding='utf-8-sig')  # Handle UTF-8 with BOM

# Clean text function
def clean_text(text):
    if not isinstance(text, str):  # Handle non-string inputs
        text = str(text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    # Keep all Tamil Unicode characters (U+0B80 to U+0BFF) and basic punctuation
    text = re.sub(r'[^\u0B80-\u0BFF\s.,!?]', '', text)
    # Normalize Tamil
    return normalizer.normalize(text.strip()) if text.strip() else ""

# Apply cleaning and print sample for debugging
df['News'] = df['News'].apply(clean_text)
print("Sample cleaned text:", df['News'].head().tolist())

# Select needed columns and rename
df_train = df[['News', 'Authenticity']].copy()
df_train.columns = ['text', 'label']

# Check label distribution
print("Label distribution (0=real, 1=fake):")
print(df_train['label'].value_counts())

# Balance if needed (optional)
if len(df_train[df_train['label'] == 0]) > len(df_train[df_train['label'] == 1]):
    df_real = df_train[df_train['label'] == 0].sample(len(df_train[df_train['label'] == 1]), random_state=42)
    df_train = pd.concat([df_real, df_train[df_train['label'] == 1]])

# Split data
train_df, temp_df = train_test_split(df_train, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Save with explicit encoding
train_df.to_csv('train.csv', index=False, encoding='utf-8-sig')
val_df.to_csv('val.csv', index=False, encoding='utf-8-sig')
test_df.to_csv('test.csv', index=False, encoding='utf-8-sig')

print(f"Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

Sample cleaned text: ['பாஸ்வேர்டை பகிரும் பயனர்களிடம் கூடுதல் கட்டணம் நெட்ஃப்ளிக்ஸ் பலே திட்டம்', 'இந்தியாவில் நடப்பு ஆண்டின் இறுதிக்குள் ஏகே ரக துப்பாக்கிகள் தயாரிப்பு?', 'பட்ஜெட் விலையில் மோட்டோ  ஸ்மார்ட்போன் இந்தியாவில் அறிமுகம்  விலை ...', 'கலாம் கண்ட கடைசி கனவை நனவாக்குவோம்', 'பட்ஜெட் விலையில் ரெட்மி ஏ ஸ்மார்ட்போன் இந்தியாவில் அறிமுகம்  விலை ...']
Label distribution (0=real, 1=fake):
label
1    2902
0    2324
Name: count, dtype: int64
Train: 4180, Val: 523, Test: 523


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from datasets import Dataset

# List of models to compare
models_to_test = {
    "XLM-RoBERTa": "xlm-roberta-base",
    "mBERT": "bert-base-multilingual-cased",
    "MuRIL": "google/muril-base-cased",
    "Distil-mBERT": "distilbert-base-multilingual-cased"
}

# Same training arguments for fair comparison
training_args = TrainingArguments(
    output_dir='./results_comparison',  # will create subfolders per model
    num_train_epochs=4,                 # 4 epochs works better for Tamil headlines
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    eval_strategy="epoch", # This was previously fixed and seems to be accepted
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to=[],  # no wandb
    logging_strategy="epoch", # Reverting to original name to fix current TypeError
    save_total_limit=2,
    seed=42,
    fp16=True,  # if you have GPU
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='binary')
    return {"accuracy": acc, "f1": f1}

results = []

print("Starting model comparison...\n")

for name, model_id in models_to_test.items():
    print(f"\n=== Training {name} ({model_id}) ===")

    # Create model-specific output dir so checkpoints don't clash
    training_args.output_dir = f"./results_{name.replace('/', '_')}"

    # Load tokenizer & model
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)

    # Tokenize function
    def tokenize_function(examples):
        return tokenizer(
            examples["text"],
            truncation=True,
            padding="max_length",
            max_length=128
        )

    # Create datasets fresh for this tokenizer
    train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
    val_ds   = Dataset.from_pandas(val_df.reset_index(drop=True))
    test_ds  = Dataset.from_pandas(test_df.reset_index(drop=True))

    train_ds = train_ds.map(tokenize_function, batched=True)
    val_ds   = val_ds.map(tokenize_function, batched=True)
    test_ds  = test_ds.map(tokenize_function, batched=True)

    # Rename label → labels (Trainer requirement) and remove text column
    train_ds = train_ds.rename_column("label", "labels").remove_columns(["text"])
    val_ds   = val_ds.rename_column("label", "labels").remove_columns(["text"])
    test_ds  = test_ds.rename_column("label", "labels").remove_columns(["text"])

    # Torch format
    train_ds.set_format("torch")
    val_ds.set_format("torch")
    test_ds.set_format("torch")

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        compute_metrics=compute_metrics,
    )

    # Train
    trainer.train()

    # Evaluate on test set
    test_results = trainer.evaluate(test_ds)

    results.append({
        "Model": name,
        "Test Accuracy": round(test_results["eval_accuracy"], 4),
        "Test F1": round(test_results["eval_f1"], 4),
        "Best Val Accuracy": round(trainer.state.best_metric, 4)
    })

    print(f"✓ {name} → Test Acc: {test_results['eval_accuracy']:.4f} | F1: {test_results['eval_f1']:.4f}\n")

# Final comparison table
import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df.sort_values("Test Accuracy", ascending=False).reset_index(drop=True)
print("\nFINAL RESULTS (sorted by Test Accuracy):")
display(results_df)

Starting model comparison...


=== Training XLM-RoBERTa (xlm-roberta-base) ===


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

...

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='Test Accuracy', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='Test F1', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(results_df['Model'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(results_df, x='Best Val Accuracy', y='Model', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [7]:
import json
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# FINAL BEST MODEL – Save it properly
best_model_name = "google/muril-base-cased" # Original model ID
muril_output_dir = "./results_MuRIL" # Directory where MuRIL checkpoints were saved

# Path to trainer_state.json
trainer_state_path = os.path.join(muril_output_dir, "trainer_state.json")

best_checkpoint_path = None
if os.path.exists(trainer_state_path):
    with open(trainer_state_path, "r") as f:
        trainer_state = json.load(f)
        best_checkpoint_path = trainer_state.get("best_model_checkpoint")

if best_checkpoint_path:
    print(f"Loading best model from checkpoint: {best_checkpoint_path}")
    # Load tokenizer from the original model ID
    tokenizer = AutoTokenizer.from_pretrained(best_model_name)

    # Load the fine-tuned model from the best checkpoint path
    model = AutoModelForSequenceClassification.from_pretrained(
        best_checkpoint_path,
        num_labels=2
    )

    # Define the final save path for the best model
    final_save_path = "./tamil_fake_news_muril_final"

    # Save the model and tokenizer to the new final directory
    model.save_pretrained(final_save_path)
    tokenizer.save_pretrained(final_save_path)

    print(f"Best model and tokenizer saved to {final_save_path}")
else:
    print(f"Error: Could not find 'best_model_checkpoint' in {trainer_state_path}. "
          "Please ensure training for MuRIL completed successfully and generated this file.")


Error: Could not find 'best_model_checkpoint' in ./results_MuRIL/trainer_state.json. Please ensure training for MuRIL completed successfully and generated this file.
